In [3]:
from dotenv import find_dotenv
import dotenv
import os

dotenv.load_dotenv(find_dotenv(filename=".env"))

os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY")
os.environ["OPENAI_API_BASE"] = os.environ.get("OPENAI_API_BASE")
os.environ["OPENAI_MODEL"] = os.environ.get("OPENAI_MODEL")

# 1. 创建 ChatModel

In [4]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(
    openai_api_key=os.environ.get("OPENAI_API_KEY"),
    openai_api_base=os.environ.get("OPENAI_API_BASE"),
    model=os.environ.get("OPENAI_MODEL"),
)

# 2. 创建 Prompt

In [14]:
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import JsonOutputParser

joke_query = "告诉我一个笑话."

parser = JsonOutputParser()

# 创建 Prompt 模板
prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template("回答用户的查询\n满满足格式为{format_instructions}\n"),
        HumanMessagePromptTemplate.from_template("{question}"),
    ],
)

# 填充 Prompt 并且声明输出解析器
prompt = prompt_template.invoke({
    "question": joke_query,
    "format_instructions": parser.get_format_instructions(),
})

print(type(prompt))

# 调用大模型获取响应
response = chat_model.invoke(prompt)

print(response.content)

<class 'langchain_core.prompt_values.ChatPromptValue'>
{
  "joke": "为什么计算机总是很冷？因为它们有很多风扇！"
}


# 3. 使用管道符

In [12]:
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import JsonOutputParser

joke_query = "告诉我一个笑话."

parser = JsonOutputParser()

# 创建 Prompt 模板
prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template("回答用户的查询\n满满足格式为{format_instructions}\n"),
        HumanMessagePromptTemplate.from_template("{question}"),
    ],
)

# 使用管道符, 将 Prompt 模板、大模型和输出解析器连接起来, 前一个输出作为后一个的输入
chain = prompt_template | chat_model | parser
response = chain.invoke({
    "question": joke_query,
    "format_instructions": parser.get_format_instructions(),
})

print(response)

{'joke': '为什么计算机总是冷？因为它们有很多风扇！'}
